---
title: "Registro Nazionale degli Aiuti di Stato"
subtitle: "_trasparenza e controllo sugli aiuti pubblici concessi in conformità alla normativa europea sugli aiuti di Stato_ "
draft: false

date: 2025-01-12
date-format: long #"MMMM DD, YYYY" # "`r format(Sys.Date(), '%B %d, %Y')`"
#date-format: "MMM D, YYYY"
created: 2025-01-12
updated: today
lang: it
tags: [agevolazioni, Invitalia, Python, RNA, Opendata]
author: "Paolo Volterra"

format:
  html:
    toc: true             # Abilita il sommario
    toc-depth: 3          # Mostra fino al terzo livello di intestazioni
    number-sections: true # Aggiunge numeri alle sezioni e sottosezioni
    toc-location: right   # Posiziona il sommario a sinistra (o usa `right` per destra)
    toc-title: "Indice"   # (Facoltativo) Titolo del sommario

categories:
  - Agevolazioni
  - Invitalia
  - Python
  - RNA
  - Opendata
  
jupyter: "python3"
execute:
  freeze: false # con true blocco l'esecuzione degli script quando aggiorno solo il testo

# 1. quarto preview d:/paolovolterra.github.io/posts/2508_RNA/RNA.qmd
# 2. quarto render d:/paolovolterra.github.io/posts/2508_RNA/RNA.qmd
# 3. git add d:/paolovolterra.github.io/docs/posts/2508_RNA/RNA.html
# 4. git status
# 5. git add . (se hai rilevato altre modifiche e le voi introitare)
# 6. git commit -m "Aggiornata la pagina: RNA"
# 7. git pull origin main --rebase
# 8. git push origin main
# 9. git log
---

# Obiettivi dello studio
- manipolazione dei parquet file
- applicazione di SQL in Python

---

Il [**Registro Nazionale degli Aiuti di Stato**](https://www.rna.gov.it/) #RNA è uno strumento gestito dal Ministero delle Imprese e del Made in Italy (ex Ministero dello Sviluppo Economico - MiSE), che permette di raccogliere, monitorare e pubblicare informazioni relative agli aiuti di Stato concessi alle imprese e agli enti in Italia. 

![](./media/01.png)

# Obiettivi principali:
1. **Trasparenza**: Pubblicare i dettagli degli aiuti concessi, garantendo che le informazioni siano accessibili a cittadini e imprese.
2. **Monitoraggio**: Assicurare che i limiti sugli aiuti stabiliti dalla normativa europea siano rispettati.
3. **Controllo**: Evitare il superamento delle soglie del regime "de minimis" e la concessione di aiuti incompatibili con le regole dell’Unione Europea.

# La realizzazione

Il #RNA è stato realizzato come parte di un'importante azione di sistema cofinanziata dall'Unione Europea. In particolare, il progetto ha beneficiato di risorse pari a **17.715.920,01 euro**, assegnate nell'ambito del **Programma Operativo Complementare al PON Governance e Capacità Istituzionale 2014-2020**. 

La progettazione e la realizzazione di una prima versione prototipale del Registro sono state completate nel 2016, con un investimento di **2.652.236,00 euro**. 

Il #RNA è operativo dal 12 agosto 2017 e rappresenta uno strumento fondamentale per garantire la trasparenza e il controllo degli aiuti di Stato concessi in Italia. 


# Informazioni contenute nel Registro:
Il #RNA include informazioni come:
- Identità del beneficiario (nome, partita IVA, codice fiscale).
- Tipo di aiuto concesso (contributo a fondo perduto, credito d’imposta, agevolazione fiscale, ecc.).
- Importo e intensità dell’aiuto.
- Riferimenti alla normativa e al regime di aiuti in base al quale è stato concesso.
- Ente erogatore.


# Normativa di riferimento:
Il #RNA è disciplinato dal **Decreto Legge n. 123/2017**, che stabilisce obblighi per le amministrazioni pubbliche e i soggetti che gestiscono misure di aiuto. L'obiettivo è garantire che tutti gli aiuti siano comunicati e registrati.

# Accesso al Registro:
Il #RNA è consultabile tramite una piattaforma online, accessibile sia dagli enti pubblici che dagli operatori economici. 

Alcune informazioni, per motivi di riservatezza, potrebbero essere soggette a limitazioni.


# Controlli sul "de minimis":
Il #RNA permette di verificare se un’impresa ha superato la soglia prevista dal regime "de minimis" (200.000 euro nell’arco di tre esercizi finanziari, salvo eccezioni per alcuni settori).

---

Oltre all'interrogazione online, #RNA permette il [download degli #opendata](https://www.rna.gov.it/open-data/aiuti)

![](./media/04.png)


# Gli opendata RNA in xml

#RNA aggiorna costantemente il proprio DB conn le segnalazioni dovute dai mittenti normativamente previsti.

Le date degli [opendata XML messi a disposizione](https://www.rna.gov.it/open-data/aiuti) corrispondono al mese della variabile "DATA_CONCESSIONE" contenuta in ognuno di loro

RNA riceve però anche informazioni datate e quindi aggiorna tutto.

Potenzialmente, quindi, **tutti** gli XML andrebbero riscaricati ...

Di seguito, ad esempio, gli ultimi 3 file parquet del 2024 da me realizzati, la data di mia aggiornamento e i df.shape

## OpenData_Aiuti_2024_12.parquet
- 20250111 (107273, 21)
- 20250105 (105454, 21)
- 20250101 (100270, 21)
- 20241221 (91504, 21)
- 20241214 (36363, 21)
- 20241207 (16243, 21)

## OpenData_Aiuti_2024_11.parquet
- 20250111 (141510, 21)
- 20250107 (141523, 21)
- 20241201 (136061, 21)
- 20241130 (115693, 21)
- 20241124 (115693, 21)
- 20241117 (19701, 21)
- 20241110 (19701, 21)

## OpenData_Aiuti_2024_10.parquet
- 20250107 (238465, 21)
- 20241110 (238076, 21)
- 20241101 (122656, 21)
- 20241029 (122656, 21)
- 20241026 (59146, 21)
- 20241019 (59146, 21)

Non ho ancora verificato se la data è quella di concessione o quella di registrazione in RNA


# Analisi sull'xml più recente

Appresso riporto alcune sintesi dell'ultimo XML -> parquet da me prodotto


In [1]:
#| echo: false
import pyarrow.parquet as pq # pip install pyarrow
import pandas as pd
import glob
import duckdb
pd.set_option('display.max_rows', None)
import os
os.chdir('D:/duckdb/files/RNA2/')
# import cudf
pd.set_option('display.max_colwidth', 500)

con = duckdb.connect()
con.execute("PRAGMA threads=8") # enable automatic query parallelization
con.execute("PRAGMA enable_object_cache") # enable caching of parquet metadata

#pd.set_option('display.max_rows', None)

pd.options.display.float_format = '{:,}'.format

# pd.options.display.float_format = '{:.2f}'.format

## Quanti aiuti contiene

In [2]:
df = con.execute("SELECT count(distinct(CF)), count((CF)), count(distinct(CAR)),count(distinct(CONCEDENTE)), sum(cast(NOMINALE as integer)), sum(cast(AIUTO as integer)) FROM 'OpenData_Aiuti_2025_01.parquet'").df()
df

,count(DISTINCT CF),count(CF),count(DISTINCT CAR),count(DISTINCT CONCEDENTE),sum(CAST(NOMINALE AS INTEGER)),sum(CAST(AIUTO AS INTEGER))
0,227772,288480,478,375,"4,198,060,832.0","2,169,603,080.0"


## Quali misure hanno più aiuti

In [3]:
df = con.execute("SELECT upper(misura) MISURA, count(CAR) CONCESSIONI, sum(cast(NOMINALE as integer)) NOMINALE, sum(cast(AIUTO as integer)) AIUTI FROM 'OpenData_Aiuti_2025_01.parquet' group by 1 order by 2 desc limit 20").df()
df

,MISURA,CONCESSIONI,NOMINALE,AIUTI
0,ESONERO DAL VERSAMENTO DEI CONTRIBUTI PREVIDENZIALI PER NUOVE ASSUNZIONI/TRASFORMAZIONI A TEMPO INDETERMINATO NEL BIENNIO 2021 - 2022 (ART. 1 COMMI 10 - 15 L. 178/2020),118427,"881,173,792.0","881,173,792.0"
1,ESONERO DAL VERSAMENTO DEI CONTRIBUTI PREVIDENZIALI PER L'ASSUNZIONE DI GIOVANI LAVORATORI ( ART. 1 COMMA 10-15 L. 178/2020 - ART. 1 COMMA 297 L. 197/2022),75758,"393,090,967.0","393,090,967.0"
2,"ESONERO DAL VERSAMENTO DEI CONTRIBUTI PREVIDENZIALI PER ASSUNZIONI DI DONNE LAVORATRICI DAL 1 LUGLIO 2022 ( ART. 1 , COMMI 16-19 L.178/2020 -ART. 1 COMMA 298 L.197/2022)",33952,"97,152,474.0","97,152,474.0"
3,ESONERO DAL VERSAMENTO DEI CONTRIBUTI PREVIDENZIALI PER ASSUNZIONI DI DONNE LAVORATRICI NEL BIENNIO 2021 - 2022 (ART. 1 COMMI 16 - 19 L. 178/2020),26226,"55,388,921.0","55,388,921.0"
4,FONDO DI GARANZIA PER LE PICCOLE E MEDIE IMPRESE,14840,"1,933,831,009.0","127,456,523.0"
5,REGOLAMENTO PER I FONDI INTERPROFESSIONALI PER LA FORMAZIONE CONTINUA PER LA CONCESSIONI DI AIUTI DI STATO ESENTATI AI SENSI DEL REGOLAMENTO CE N.651/2014 E IN REGIME DE MINIMIS AI SENSI DEL REGOLAMENTO CE N.1407/2013,6093,"30,887,740.0","30,476,607.0"
6,"NUOVA SABATINI - FINANZIAMENTI PER L'ACQUISTO DI NUOVI MACCHINARI, IMPIANTI E ATTREZZATURE DA PARTE DELLE PICCOLE E MEDIE IMPRESE - VERSIONE MODIFICATA DA CODICE CE SA.47180",2502,"36,229,375.0","36,229,375.0"
7,VOUCHER DIGITALIZZAZIONE 2024,536,"8,670,819.0","8,670,819.0"
8,PROGETTI INTEGRATI DI FILIERA 2023-2025,449,"14,386,187.0","14,386,187.0"
9,AVVISO PUBBLICO PER L’ASSEGNAZIONE DI VOUCHER AZIENDALI A CATALOGO PER INTERVENTI DI FORMAZIONE CONTINUA – SECONDA EDIZIONE,332,"1,051,790.0","1,051,790.0"


## Quali concedenti hanno più aiuti

purtroppo il nome del concedente è scritto in modi diversi

In [4]:
df = con.execute("SELECT upper(concedente) CONCEDENTE, count(CAR) CONCESSIONI, sum(cast(NOMINALE as integer)) NOMINALE, sum(cast(AIUTO as integer)) AIUTI FROM 'OpenData_Aiuti_2025_01.parquet' group by 1 order by 2 desc limit 10").df()
df

,CONCEDENTE,CONCESSIONI,NOMINALE,AIUTI
0,INPS,254432,"1,427,160,166.0","1,427,160,166.0"
1,BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A.,14914,"1,934,188,400.0","127,813,914.0"
2,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - DIPARTIMENTO PER LE POLITICHE PER LE IMPRESE - DIREZIONE GENERALE PER GLI INCENTIVI ALLE IMPRESE,2796,"125,965,619.0","89,876,791.0"
3,FONARCOM,1529,"2,752,960.0","2,752,960.0"
4,FONDER,1117,"1,710,921.0","1,618,699.0"
5,FONDO FOR.TE,768,"4,851,253.0","4,851,253.0"
6,FONDOPROFESSIONI,606,"579,084.0","575,934.0"
7,UNIONE REGIONALE DELLE CAMERE DI COMMERCIO INDUSTRIA ARTIGIANATO AGRICOLTURA DEL PIEMONTE,560,"8,765,133.0","8,765,133.0"
8,FONDIMPRESA,552,"7,712,030.0","7,712,030.0"
9,FONCOOP,531,"5,685,118.0","5,685,118.0"


## Gli aiuti in ordine di valore nominale €

In [5]:
df = con.execute("SELECT upper(concedente) CONCEDENTE, count(CAR) CONCESSIONI, sum(cast(NOMINALE as integer)) NOMINALE, sum(cast(AIUTO as integer)) AIUTI FROM 'OpenData_Aiuti_2025_01.parquet' group by 1 order by 3 desc limit 10").df()
df

,CONCEDENTE,CONCESSIONI,NOMINALE,AIUTI
0,BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A.,14914,"1,934,188,400.0","127,813,914.0"
1,INPS,254432,"1,427,160,166.0","1,427,160,166.0"
2,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,48,"192,359,736.0","152,555,160.0"
3,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - DIPARTIMENTO PER LE POLITICHE PER LE IMPRESE - DIREZIONE GENERALE PER GLI INCENTIVI ALLE IMPRESE,2796,"125,965,619.0","89,876,791.0"
4,INVITALIA - AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA S.P.A.,299,"102,743,941.0","78,368,900.0"
5,REGIONE PUGLIA - DIPARTIMENTO SVILUPPO ECONOMICO,224,"20,505,184.0","20,505,184.0"
6,REGIONE AUTONOMA FRIULI VENEZIA GIULIA - DIREZIONE CENTRALE ATTIVITÀ PRODUTTIVE,66,"18,257,297.0","3,121,015.0"
7,INAIL - DIREZIONE CENTRALE PREVENZIONE,195,"17,095,554.0","13,841,898.0"
8,REGIONE PIEMONTE - DIREZIONE COORDINAMENTO POLITICHE E FONDI EUROPEI,449,"14,386,187.0","14,386,187.0"
9,REGIONE PIEMONTE - DIREZIONE AGRICOLTURA E CIBO,21,"14,261,705.0","370,163.0"


## valori raggruppati per data 

In [6]:
df = con.execute("SELECT CONCESSIONE, count(CAR) CONCESSIONI, sum(cast(NOMINALE as integer)) NOMINALE, sum(cast(AIUTO as integer)) AIUTI FROM 'OpenData_Aiuti_2025_01.parquet' group by 1 order by 1 desc").df()
df

,CONCESSIONE,CONCESSIONI,NOMINALE,AIUTI
0,2025-01-31,45421,"561,196,693.0","278,277,081.0"
1,2025-01-30,36476,"208,556,936.0","202,515,587.0"
2,2025-01-29,35686,"132,254,858.0","128,777,052.0"
3,2025-01-28,11855,"369,797,160.0","125,641,874.0"
4,2025-01-27,43586,"350,755,398.0","343,786,019.0"
5,2025-01-26,9,"2,704.0","2,704.0"
6,2025-01-25,44938,"365,476,683.0","365,425,043.0"
7,2025-01-24,40995,"561,494,057.0","264,903,743.0"
8,2025-01-23,12281,"82,569,511.0","54,672,683.0"
9,2025-01-22,630,"28,215,232.0","23,393,064.0"


### valori raggruppati per regione 

In [7]:
df = con.execute("SELECT upper(REGIONE) REGIONE, count(CAR), sum(cast(NOMINALE as integer)), sum(cast(AIUTO as integer)) FROM 'OpenData_Aiuti_2025_01.parquet' group by 1 order by 3 desc limit 10").df()
df

,REGIONE,count(CAR),sum(CAST(NOMINALE AS INTEGER)),sum(CAST(AIUTO AS INTEGER))
0,LOMBARDIA,49892,"873,593,711.0","545,073,929.0"
1,CAMPANIA,31179,"392,387,204.0","172,188,255.0"
2,VENETO,24454,"372,076,854.0","179,851,953.0"
3,PIEMONTE,17998,"326,266,290.0","151,577,391.0"
4,LAZIO,22051,"316,076,119.0","163,111,372.0"
5,EMILIA-ROMAGNA,20436,"295,503,890.0","168,523,740.0"
6,PUGLIA,21462,"275,952,282.0","128,651,247.0"
7,SICILIA,24128,"258,160,221.0","113,920,615.0"
8,TOSCANA,16063,"224,343,793.0","106,355,032.0"
9,FRIULI-VENEZIA GIULIA,6176,"127,429,098.0","80,238,575.0"


### valori raggruppati per ATECO 

In [8]:
df = con.execute("SELECT upper(SETTORE_ATTIVITA) SETTORE_ATTIVITA, count(CAR), sum(cast(NOMINALE as integer)), sum(cast(AIUTO as integer)) FROM 'OpenData_Aiuti_2025_01.parquet' group by 1 order by 3 desc limit 10").df()
df

,SETTORE_ATTIVITA,count(CAR),sum(CAST(NOMINALE AS INTEGER)),sum(CAST(AIUTO AS INTEGER))
0,F.41.2,9287,"163,293,794.0","48,573,220.0"
1,D.35.1,523,"148,851,639.0","80,323,869.0"
2,I.56.1,20292,"144,082,919.0","82,794,010.0"
3,F.43.2,10835,"126,596,444.0","59,673,120.0"
4,J.62.0,6811,"117,881,630.0","83,690,826.0"
5,H.49.4,5105,"117,519,650.0","35,566,283.0"
6,G.47.7,13507,"97,903,520.0","57,478,971.0"
7,D.35.2,97,"94,972,219.0","90,234,671.0"
8,G.46.4,4646,"94,557,197.0","27,224,214.0"
9,I.55.1,3550,"88,270,063.0","36,006,125.0"


### valori raggruppati per strumento 

In [9]:
df = con.execute("SELECT upper(DES_STRUMENTO) STRUMENTO, count(CAR), sum(cast(NOMINALE as integer)), sum(cast(AIUTO as integer)) FROM 'OpenData_Aiuti_2025_01.parquet' group by 1 order by 2 desc limit 10").df()
df

,STRUMENTO,count(CAR),sum(CAST(NOMINALE AS INTEGER)),sum(CAST(AIUTO AS INTEGER))
0,AGEVOLAZIONE FISCALE O ESENZIONE FISCALE,254746,"1,442,732,051.0","1,442,123,047.0"
1,SOVVENZIONE/CONTRIBUTO IN CONTO INTERESSI,16696,"544,531,517.0","449,595,824.0"
2,GARANZIA (SE DEL CASO CON UN RIFERIMENTO ALLA DECISIONE DELLA COMMISSIONE (10)),15473,"1,976,479,946.0","129,662,953.0"
3,PRESTITO/ANTICIPO RIMBORSABILE,495,"106,765,282.0","26,157,762.0"
4,SCONTO PER PARTECIPAZIONE A INIZIATIVE PROGRAMMATE (CATEGORIA: SOVVENZIONE/CONTRIBUTO IN CONTO INTERESSI),449,"14,386,187.0","14,386,187.0"
5,EROGAZIONE SERVIZI (CATEGORIA: SOVVENZIONE/CONTRIBUTO IN CONTO INTERESSI),188,"2,729,160.0","2,176,276.0"
6,MISURA PER IL FINANZIAMENTO DEL RISCHIO,103,"9,039,541.0","7,376,896.0"
7,CONTO CAPITALE A FONDO PERDUTO (CATEGORIA: MISURA PER IL FINANZIAMENTO DEL RISCHIO),71,"6,646,324.0","5,209,815.0"
8,CONTRIBUTO A FONDO PERDUTO (CATEGORIA: SOVVENZIONE/CONTRIBUTO IN CONTO INTERESSI),62,"4,285,651.0","3,004,296.0"
9,SCONTO SUL CANONE DI CONNETTIVITÀ (CATEGORIA: SOVVENZIONE/CONTRIBUTO IN CONTO INTERESSI),45,"70,750.0","70,750.0"


### gli importi più grandi 

In [10]:
df = con.execute("SELECT IMPRESA, REGIONE, CONCEDENTE, MISURA, NOMINALE, AIUTO FROM 'OpenData_Aiuti_2025_01.parquet' order by 5 desc limit 20").df()
df

,IMPRESA,REGIONE,CONCEDENTE,MISURA,NOMINALE,AIUTO
0,BIOWASTE CH4 LEGNANO S.R.L.,Lombardia,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Decreto 2 marzo 2018: Promozione dell’uso del biometano e degli altri biocarburanti avanzati nel settore dei trasporti (GU n.65 del 19/3/2018),"32,489,271.33","32,489,271.33"
1,ENIBIOCH4IN QUADRUVIUM - S.R.L. SOCIETA' AGRICOLA,Friuli-Venezia Giulia,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Decreto 2 marzo 2018: Promozione dell’uso del biometano e degli altri biocarburanti avanzati nel settore dei trasporti (GU n.65 del 19/3/2018),"27,020,243.99","27,020,243.99"
2,AMBIENTE & RISORSE SRL,Lombardia,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Decreto 2 marzo 2018: Promozione dell’uso del biometano e degli altri biocarburanti avanzati nel settore dei trasporti (GU n.65 del 19/3/2018),"25,720,673.14","25,720,673.14"
3,ENGIE SERVIZI S.P.A.,Lazio,REGIONE LAZIO,Regime di aiuti agli investimenti per la produzione di idrogeno rinnovabile in aree industriali dismesse – PNRR M2C2 Investimento 3.1,"12,000,000.0","12,000,000.0"
4,SOCIETA' AGRICOLA LORENZO S.S. DI ODETTO ALEX E LUCA,Piemonte,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Incentivazione dell'energia prodotta da impianti a biogas da reflui e materie derivanti dalle aziende agricole realizzatrici,"10,122,866.0","5,568,880.0"
5,RANDSTAD ITALIA S.P.A.,Lombardia,INPS,Esonero dal versamento dei contributi previdenziali per nuove assunzioni/trasformazioni a tempo indeterminato nel biennio 2021 - 2022 (art. 1 commi 10 - 15 L. 178/2020),"9,490,693.25","9,490,693.25"
6,SAN GIORGIO DI CONTI GIORGIO & C. S.S. SOCIETA' AGRICOLA,Lombardia,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Incentivazione dell'energia prodotta da impianti a biogas da reflui e materie derivanti dalle aziende agricole realizzatrici,"8,859,626.0","4,873,935.0"
7,AZ.AGRIC. CORTAZZA DI RIBONI GIOVANNI E LUIGI SOCIETA' AGRICOLA S S.,Lombardia,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Incentivazione dell'energia prodotta da impianti a biogas da reflui e materie derivanti dalle aziende agricole realizzatrici,"8,859,626.0","4,873,935.0"
8,SOCIETA' AGRICOLA LE GAZZE,Lombardia,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Incentivazione dell'energia prodotta da impianti a biogas da reflui e materie derivanti dalle aziende agricole realizzatrici,"8,859,626.0","4,873,935.0"
9,BELLINI ANTONIO E FACCHI ANNA SOCIETA' AGRICOLA S.S.,Lombardia,GSE - GESTORE SERVIZI ENERGETICI S.P.A.,Incentivazione dell'energia prodotta da impianti a biogas da reflui e materie derivanti dalle aziende agricole realizzatrici,"8,859,626.0","4,873,935.0"


## Analisi per concedente

Il nome del concedente viene scritto in modi diversi; non c' il suo codice fiscale o la sua P.IVA

### Un esempio: gli aiuti concessi da Invitalia
- INVITALIA - AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA S.P.A.
- AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA SPA - INVITALIA (05678721001)


In [11]:
df = con.execute("SELECT upper(misura) MISURA, count(CAR) CONCESSIONI, sum(cast(NOMINALE as integer)) NOMINALE, sum(cast(AIUTO as integer)) AIUTI FROM 'OpenData_Aiuti_2025_01.parquet' where concedente like '%INVI%' group by 1 order by 3 desc limit 10").df()
df

,MISURA,CONCESSIONI,NOMINALE,AIUTI
0,INVESTIMENTI SOSTENIBILI 4.0 (PN RIC 2021-2027),68,"50,350,162.0","32,502,255.0"
1,"CONTRATTI DI SVILUPPO DI CUI ALL'ARTICOLO 43 DEL DECRETO-LEGGE 25 GIUGNO 2008, N.112, CONVERTITO DALLA LEGGE 6 AGOSTO 2008, N. 133 (ADEGUAMENTO GBER E PROROGA 2026)",6,"18,240,078.0","18,240,078.0"
2,FONDO PER IL SOSTEGNO ALLA TRANSIZIONE INDUSTRIALE,10,"12,759,588.0","12,759,588.0"
3,4:2:5 FONDO ROTATIVO IMPRESE (FRI) PER IL SOSTEGNO ALLE IMPRESE E GLI INVESTIMENTI DI SVILUPPO,39,"8,856,020.0","8,144,283.0"
4,NIT0 - NUOVE IMPRESE A TASSO ZERO,60,"7,021,985.0","3,466,904.0"
5,CONTRATTI DI SVILUPPO AGROINDUSTRIALI - 2023,1,"5,659,164.0","5,659,164.0"
6,"PNRR, M1C3 INVESTIMENTI 4.2.1; 4.2.2 AVVISO PUBBLICO EX ARTT. 1 E 4 DL 152/2021",79,"5,176,411.0","4,636,411.0"
7,SMART&START - RIORDINO DEGLI INTERVENTI DI SOSTEGNO ALLA NASCITA E ALLO SVILUPPO DI START-UP INNOVATIVE IN TUTTO IL TERRITORIO NAZIONALE - VERSIONE MODIFICATA DAL CODICE CE SA.55753,13,"4,243,242.0","2,056,740.0"
8,RESTO AL SUD,84,"2,654,480.0","2,612,698.0"
9,"MAGGIORAZIONE DEL CONTRIBUTO PER L'ACQUISTO DI VEICOLI DI CATEGORIA M1 DA PARTE DEI TITOLARI DI LICENZE DI TAXI E DI SOGGETTI AUTORIZZATI ALL'ESERCIZIO DEL SERVIZIO DI NOLEGGIO CON CONDUCENTE (""ECOBONUS TAXI"")",29,"963,429.0","358,000.0"


### valori raggruppati per data 

In [12]:
df = con.execute("SELECT CONCESSIONE, count(CAR) CONCESSIONI, sum(cast(NOMINALE as integer)) NOMINALE, sum(cast(AIUTO as integer)) AIUTI FROM 'OpenData_Aiuti_2025_01.parquet' where concedente like '%INVI%' group by 1 order by 1 desc").df()
df

,CONCESSIONE,CONCESSIONI,NOMINALE,AIUTI
0,2025-01-31,32,"12,053,460.0","9,138,731.0"
1,2025-01-30,31,"2,126,257.0","1,895,920.0"
2,2025-01-29,2,"18,000.0","18,000.0"
3,2025-01-28,46,"21,511,981.0","13,967,487.0"
4,2025-01-24,22,"3,362,163.0","3,357,630.0"
5,2025-01-23,8,"1,808,445.0","1,699,898.0"
6,2025-01-22,47,"18,047,651.0","16,968,552.0"
7,2025-01-21,13,"5,512,491.0","5,046,747.0"
8,2025-01-20,1,"28,465.0","22,772.0"
9,2025-01-17,6,"6,713,997.0","6,601,489.0"
